# Assignment 9: Bayesian Analysis

## Greg DeVore
## ML210
## March 10th, 2018

Dataset needed: Tennis weather from within (https://computersciencesource.wordpress.com/2010/01/28/year-2-machine-learning-naive-bayes-classifier/)

## Naïve Bayes

Train a Naïve Bayes model on the datset that appears as a table in the referenced URL.

The data is embedded in a table on the web page listed above. Rather than doing a simple copy paste, let's experiment with web scraping to extract the table directly.

In [54]:
# Load rvest package
require(rvest)
tennis_page <- 'https://computersciencesource.wordpress.com/2010/01/28/year-2-machine-learning-naive-bayes-classifier/'

# Extract table from site
table_data <- tennis_page %>% read_html() %>% 
  html_nodes(xpath = '//*[@id="content-left"]/div[2]/div[1]/table[1]') %>% 
    html_table()
# Grab data frame
tennis_data <- table_data[[1]]
# Inspect table
head(tennis_data)

X1,X2,X3,X4,X5,X6
Day,Outlook,Temperature,Humidity,Wind,Play Tennis?
1,Sunny,Hot,High,Weak,No
2,Sunny,Hot,High,Strong,No
3,Overcast,Hot,High,Weak,Yes
4,Rain,Mild,High,Weak,Yes
5,Rain,Cool,Normal,Weak,Yes


This is almost what we want, except that the column headers are actually the first row of the data frame. Let's fix this and reinspect the table.

In [55]:
library(dplyr)
# Extract first row for column names
col.names <- unlist(tennis_data[1,])
# Change last column (response) to 'Play'
col.names[length(col.names)] <- 'Play'
# Rename columns
names(tennis_data) <- col.names
# Remove first row and renumber rows
tennis_data <- tennis_data[-1, ]
row.names(tennis_data) <- as.numeric(tennis_data$Day)
# Finally, remove 'Day' feature
tennis_data <- tennis_data %>% select(-Day)
# Convert to factors
tennis_data[, ] <- lapply(tennis_data[, ], as.factor)
tennis_data

Outlook,Temperature,Humidity,Wind,Play
Sunny,Hot,High,Weak,No
Sunny,Hot,High,Strong,No
Overcast,Hot,High,Weak,Yes
Rain,Mild,High,Weak,Yes
Rain,Cool,Normal,Weak,Yes
Rain,Cool,Normal,Strong,No
Overcast,Cool,Normal,Strong,Yes
Sunny,Mild,High,Weak,No
Sunny,Cool,Normal,Weak,Yes
Rain,Mild,Normal,Weak,Yes


This looks much better, and now we're ready to train a Naive Bayes model. We'll use the 'e1071' package to build our model.

In [56]:
require(e1071)
tennis.nb <- naiveBayes(Play ~ ., data = tennis_data)
print(tennis.nb)


Naive Bayes Classifier for Discrete Predictors

Call:
naiveBayes.default(x = X, y = Y, laplace = laplace)

A-priori probabilities:
Y
       No       Yes 
0.3571429 0.6428571 

Conditional probabilities:
     Outlook
Y      Overcast      Rain     Sunny
  No  0.0000000 0.4000000 0.6000000
  Yes 0.4444444 0.3333333 0.2222222

     Temperature
Y          Cool       Hot      Mild
  No  0.2000000 0.4000000 0.4000000
  Yes 0.3333333 0.2222222 0.4444444

     Humidity
Y          High    Normal
  No  0.8000000 0.2000000
  Yes 0.3333333 0.6666667

     Wind
Y        Strong      Weak
  No  0.6000000 0.4000000
  Yes 0.3333333 0.6666667



Note that all conditional probabilities sum to 1 (across columns), which is expected. Also note that the naive assumption in a Naive Bayes model is rarely valid, as it is assumed that all of the features are conditionally independent of one another given the value of the response (in this case, whether tennis was played or not). In reality, since we're dealing with weather, this assumption isn't strictly valid. For example, if the temperature is 'Hot', you are more likely to observe an outlook of 'Sunny' than 'Rainy'. Similarly, if the temperature is 'Cool', you are more likely to observe a humidity of 'Normal' than 'High'. Despite these violations, Naive Bayes models can still provide meaningful results.

Let's see how the model does on predicting the training data.

In [57]:
require(caret)
tennis.preds <- predict(tennis.nb, newdata = tennis_data, type = 'class')
confusionMatrix(tennis.preds,tennis_data$Play,positive = 'Yes')

Confusion Matrix and Statistics

          Reference
Prediction No Yes
       No   4   0
       Yes  1   9
                                          
               Accuracy : 0.9286          
                 95% CI : (0.6613, 0.9982)
    No Information Rate : 0.6429          
    P-Value [Acc > NIR] : 0.01807         
                                          
                  Kappa : 0.8372          
 Mcnemar's Test P-Value : 1.00000         
                                          
            Sensitivity : 1.0000          
            Specificity : 0.8000          
         Pos Pred Value : 0.9000          
         Neg Pred Value : 1.0000          
             Prevalence : 0.6429          
         Detection Rate : 0.6429          
   Detection Prevalence : 0.7143          
      Balanced Accuracy : 0.9000          
                                          
       'Positive' Class : Yes             
                                          

Overall, the model does a good job of predicting the training data, which it should. The accuracy is ~93%, and the true positive and false positive rates are 100% and 20%, respectively. Only one training observation was incorrectly classified.

As one last check, let's create a few new observations and predict whether a match will be played.

In [58]:
# Create observations for three different days, one nice and two not-so-nice
test.outlook <- factor(c('Sunny','Sunny','Rain'),
                        levels = levels(tennis_data$Outlook))
test.temperature <- factor(c('Mild','Hot','Cool'),
                        levels = levels(tennis_data$Temperature))
test.humidity <- factor(c('Normal','High','Normal'),
                        levels = levels(tennis_data$Humidity))
test.wind <- factor(c('Weak','Weak','Strong'),
                        levels = levels(tennis_data$Wind))
# Create a new data frame
test.data <- data.frame(Outlook = test.outlook, 
                        Temperature = test.temperature,
                        Humidity = test.humidity, 
                        Wind = test.wind)
cat('Test cases:')
predict.test <- predict(tennis.nb, newdata = test.data, type = 'raw')
cbind(test.data,predict.test)

Test cases:

Outlook,Temperature,Humidity,Wind,No,Yes
Sunny,Mild,Normal,Weak,0.1954948,0.8045052
Sunny,Hot,High,Weak,0.7954173,0.2045827
Rain,Cool,Normal,Strong,0.1776316,0.8223684


The first two results are intuitive, namely that tennis would be played on a sunny, mild day with low humidity and low wind, and that tennis wouldn't be played on a sunny, hot day with high humidity. The third result, on the 
other hand, is less intuitive. It seems that a cool, rainy day with high winds would be a bad day for tennis, however the model says we should play. 

Recall that there was one observation in the training data that was incorrectly classified when it was run through the model. Let's take a closer look:

In [59]:
# Find all observations that were misclassified
idx <- which(tennis.preds != tennis_data$Play)
tennis_data[idx, ]

,Outlook,Temperature,Humidity,Wind,Play
6,Rain,Cool,Normal,Strong,No


This is in fact the same test observation we created to simulate a not so nice day. In the training data, tennis was not played on this day, which makes sense, but the model predicted that it would. In this case, perhaps Naive Bayes is not the best approach, or we may be suffering from a lack of training data (there were only 14 observations).

##  Bayesian Nets

Shingles is a possible cause of nerve damage and is also an explanation for increased blood pressure. In turn, either of these could cause a stroke. Severe headaches could also be explained by nerve damage.

a: shingles
b: increased blood pressure
c: nerve damage
d: stroke
e: severe headaches

Create a Bayesian net that captures these relationships.

Use these probabilities:  P(a)=0.2,P(b|a)=0.75,P(b|¬a) = 0.25,P(c|a)=0.2,Pr(c|¬a)=0.05, P(e|c)=0.8,Pr(e|¬c)=0.6, P (d|b∧c)=0.8,P(d|b∧ ¬c)=0.8,P(d|¬b ∧ c)=0.8,P(d|¬b ∧ ¬c)=0.05

Train this dataset. Predict value of ) P(shingles|¬severe headaches)

In [60]:
require(bnlearn)
# Structure is as follows:
# A - Parent
# B & C - Children of A
# D - Child of B and C
# E - Child of C
# Network model
net <-  model2network("[A][B|A][C|A][D|B:C][E|C]")
# Define conditional probabilities for the nodes
YN <- c('Yes','No')
# Shingles
# Entires of the matrix are read as P(A) and P(-A)
cptA <- matrix(c(0.2,0.8), ncol = 2, dimnames = list(NULL,YN))
# Blood Pressure
# First column corresponds to B conditioned on A = Yes
# Second column corresponds to B conditioned on A = No
# Columns must always sum to 1!
cptB <- matrix(c(0.75, 0.25, 0.25, 0.75), ncol = 2, 
               dimnames = list('B' = YN, 'A' = YN))
# Nerve Damage
cptC <- matrix(c(0.2, 0.8, 0.05, 0.95), ncol = 2,
               dimnames = list('C' = YN, 'A' = YN))
# Severe Headaches
cptE <- matrix(c(0.8, 0.2, 0.6, 0.4), ncol = 2,
               dimnames = list('E' = YN, 'C' = YN))
# Stroke
# D depends on B and C, so we create a 3-D matrix here with 'slices' for C
# Could also have done this with slices for B
cptD <- c(0.8, 0.2, 0.8, 0.2, 0.8, 0.2, 0.05, 0.95)
dim(cptD) <- c(2,2,2)
dimnames(cptD) <- list('D' = YN, 'B' = YN, 'C' = YN)

# Create bn.fit model using the custom tables
shingles.net <- custom.fit(net, dist = list(A = cptA, B = cptB, C = cptC,
                                            D = cptD, E = cptE))
# View the model
shingles.net


  Bayesian network parameters

  Parameters of node A (multinomial distribution)

Conditional probability table:
 
Yes  No 
0.2 0.8 

  Parameters of node B (multinomial distribution)

Conditional probability table:
 
     A
B      Yes   No
  Yes 0.75 0.25
  No  0.25 0.75

  Parameters of node C (multinomial distribution)

Conditional probability table:
 
     A
C      Yes   No
  Yes 0.20 0.05
  No  0.80 0.95

  Parameters of node D (multinomial distribution)

Conditional probability table:
 
, , C = Yes

     B
D      Yes   No
  Yes 0.80 0.80
  No  0.20 0.20

, , C = No

     B
D      Yes   No
  Yes 0.80 0.05
  No  0.20 0.95


  Parameters of node E (multinomial distribution)

Conditional probability table:
 
     C
E     Yes  No
  Yes 0.8 0.6
  No  0.2 0.4


The conditional probability tables above represent the Bayesian network form of the probabilities specified at the beginning of the problem.

Before making the requested prediction, let's test the model using some known probabilities.

In [61]:
# Start with conditional probabilities from training
case <- c('P(A)','P(B|-A)','P(D|-B^-C)')
predicted <- vector(length = 3)
# P(A), should return 0.2
predicted[1] <- cpquery(shingles.net, (A == 'Yes'), TRUE)
# P(B|-A), should return 0.25
predicted[2] <- cpquery(shingles.net, (B == 'Yes'), (A == 'No'))
# P(D|-B ^ -C), should return 0.05
predicted[3] <- cpquery(shingles.net, (D == 'Yes'), 
                                       (B == 'No' & C == 'No'))
actual <- c(0.2, 0.25, 0.05)
delta = abs(predicted - actual)
unit.test = data.frame(case = case,
                       predicted = predicted, 
                       actual = actual, 
                       delta = delta)
unit.test

case,predicted,actual,delta
P(A),0.2050000,0.20,0.005000000
P(B|-A),0.2512500,0.25,0.001250000
P(D|-B^-C),0.0530303,0.05,0.003030303


The model seems to be accurately representing the underlying probabilities used to build the network. Let's go ahead and predict the requested probability of the event of having shingles, given the absense of severe headaches.

In [62]:
# Predict P(A|-E)
cat('The probability of having shingles given that 
severe headaches are NOT present is:')
cpquery(shingles.net, (A == 'Yes'), (E == 'No'))

The probability of having shingles given that 
severe headaches are NOT present is:

[1] 0.1703267

There is just under a 20% that a patient has shingles, given that they are not presenting with severe headache.